In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/data.csv")
df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
93807,610,166534,4.0
93808,610,168248,5.0
93809,610,168250,5.0
93810,610,168252,5.0


# VISUALIZATIONS HERE PLS BLYAT

**hudsiahdiuashdiuashdiahsdui**

In [3]:
genres = pd.read_csv("data/movies.csv")["genres"].str.split("|").tolist()
genres = set([genre for sublist in genres for genre in sublist])
genres, len(genres)

({'(no genres listed)',
  'Action',
  'Adventure',
  'Animation',
  'Children',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Fantasy',
  'Film-Noir',
  'Horror',
  'IMAX',
  'Musical',
  'Mystery',
  'Romance',
  'Sci-Fi',
  'Thriller',
  'War',
  'Western'},
 20)